[辻真吾・矢吹太朗『ゼロからはじめるデータサイエンス入門』（講談社, 2021）](https://github.com/taroyabuki/fromzero)

In [ ]:
# Google Colaboratoryの環境設定
if (Sys.getenv("COLAB_JUPYTER_IP") != "") {
  options(Ncpus = parallel::detectCores())
  installed_packages <- rownames(installed.packages())
  packages_to_install <- c("exactci", "ggmosaic", "pastecs", "psych", "vcd")
  install.packages(setdiff(packages_to_install, installed_packages))
}

## 4.1 記述統計

In [ ]:
x <- c(165, 170, 175, 180, 185)
mean(x) # 平均

In [ ]:
n <- length(x) # サンプルサイズ
sum(x) / n

In [ ]:
y <- c(173, 174, 175, 176, 177)
mean(y)

In [ ]:
var(x) # xの分散

var(y) # yの分散

In [ ]:
sum((x - mean(x))^2) / (n - 1)

In [ ]:
sd(x) # xの標準偏差

sd(y) # yの標準偏差

In [ ]:
var(x)**0.5 # xの標準偏差

In [ ]:
psych::describe(x)

# あるいは

pastecs::stat.desc(x)

In [ ]:
quantile(x)

In [ ]:
x <- c(165, 170, 175, 180, 185)

var(x)                # 不偏分散

mean((x - mean(x))^2) # 標本分散
# あるいは
n <- length(x)
var(x) * (n - 1) / n  # 標本分散

In [ ]:
sd(x)                     # √不偏分散

mean((x - mean(x))^2)^0.5 # √標本分散
# あるいは
sd(x) * sqrt((n - 1) / n) # √標本分散

In [ ]:
sd(x) / length(x)**0.5

In [ ]:
library(tidyverse)

my_df <- data.frame(
  name    = c("A", "B", "C", "D"),
  english = c( 60,  90,  70,  90),
  math    = c( 70,  80,  90, 100),
  gender  = c("f", "m", "m", "f"))

In [ ]:
var(my_df$english)

In [ ]:
# 結果はベクタ
my_df[, c(2, 3)] %>% sapply(var)

# 結果はリスト
my_df[, c(2, 3)] %>% lapply(var)

# 結果はデータフレーム
my_df[, c(2, 3)] %>% # 2, 3列目
  summarize(across(  # の
    everything(),    # 全ての
    var))            # 不偏分散
# あるいは
my_df %>%              # データフレーム
  summarize(across(    # の
    where(is.numeric), # 数値の列の
    var))              # 不偏分散
# あるいは
my_df %>%              # データフレーム
  summarize(across(    # の
    where(is.numeric), # 数値の列の
    function(x) { var(x) })) # 不偏分散


In [ ]:
psych::describe(my_df)

# あるいは

pastecs::stat.desc(my_df)
# 以下省略

In [ ]:
table(my_df$gender)


In [ ]:
my_df2 <- data.frame(
  gender = my_df$gender,
  excel = my_df$math >= 80)
table(my_df2)


In [ ]:
my_df %>% group_by(gender) %>%
  summarize(across(
    where(is.numeric), mean),
    .groups = "drop") # グループ化解除


## 4.2 データの可視化

In [ ]:
head(iris)

In [ ]:
hist(iris$Sepal.Length)

In [ ]:
x <- c(10, 20, 30)
hist(x, breaks = 2) # 階級数は2

In [ ]:
x <- iris$Sepal.Length
tmp <- seq(min(x), max(x),
           length.out = 10)
hist(x, breaks = tmp, right = FALSE)

In [ ]:
plot(iris$Sepal.Length,
     iris$Sepal.Width)

In [ ]:
boxplot(iris[, -5])

In [ ]:
library(tidyverse)
my_df <- psych::describe(iris[, -5])
my_df %>% select(mean, sd, se)

In [ ]:
tmp <- rownames(my_df)
my_df %>% ggplot(aes(x = factor(tmp, levels = tmp), y = mean)) +
  geom_col() +
  geom_errorbar(aes(ymin = mean - se, ymax = mean + se)) +
  xlab(NULL)

In [ ]:
my_group <- iris %>% group_by(Species)       # 品種ごとに，

my_df <- my_group %>%                        # 各変数の，平均と
  summarize(across(everything(), mean)) %>%
  pivot_longer(-Species)

tmp <- my_group %>%                          # 標準誤差を求める．
  summarize(across(everything(), ~ sd(.) / length(.)**0.5)) %>%
  pivot_longer(-Species)

my_df$se <- tmp$value
head(my_df)

In [ ]:
my_df %>%
  ggplot(aes(x = Species, y = value, fill = name)) +
  geom_col(position = "dodge") +
  geom_errorbar(aes(ymin = value - se, ymax = value + se), position = "dodge")

In [ ]:
# 各変数の平均
iris %>% pivot_longer(-Species) %>%
  ggplot(aes(x = name, y = value)) +
  geom_bar(stat = "summary", fun = mean) +
  stat_summary(geom = "errorbar", fun.data = mean_se) +
  xlab(NULL)

# 各変数の平均（品種ごと）
iris %>% pivot_longer(-Species) %>%
  ggplot(aes(x = Species, y = value, fill = name)) +
  geom_bar(stat = "summary", fun = mean, position = "dodge") +
  stat_summary(geom = "errorbar", fun.data = mean_se, position = "dodge")

In [ ]:
my_df <- data.frame(
  Species = iris$Species,
  w_Sepal = iris$Sepal.Width > 3)
table(my_df) # 分割表

mosaicplot(
  formula = ~ Species + w_Sepal,
  data = my_df)

In [ ]:
library(vcd)
vcd::mosaic(formula = ~w_Sepal + Species, data = my_df,
            labeling = labeling_values)

In [ ]:
curve(x^3 - x, -2, 2)

In [ ]:
x <- iris$Sepal.Length
tmp <- seq(min(x), max(x),
           length.out = 10)
iris %>%
  ggplot(aes(x = Sepal.Length)) +
  geom_histogram(breaks = tmp,
                 closed = "left")

In [ ]:
iris %>%
  ggplot(aes(x = Sepal.Length,
             y = Sepal.Width)) +
  geom_point()

In [ ]:
iris %>%
  pivot_longer(-Species) %>%
  ggplot(aes(
    x = factor(name,
               levels = names(iris)),
    y = value)) +
  geom_boxplot() +
  xlab(NULL)

In [ ]:
library(ggmosaic)
my_df <- data.frame(
  Species = iris$Species,
  w_Sepal = iris$Sepal.Width > 3)
my_df %>%
  ggplot() +
  geom_mosaic(
    aes(x = product(w_Sepal, Species)))

In [ ]:
f <- function(x) { x^3 - x }
data.frame(x = c(-2, 2)) %>%
  ggplot(aes(x = x)) +
  stat_function(fun = f)

## 4.3 乱数

In [ ]:
x <- sample(x = 1:6,        # 範囲
            size = 10000,   # 乱数の数
            replace = TRUE) # 重複あり
hist(x, breaks = 0:6) # ヒストグラム

In [ ]:
x <- runif(min = 0,  # 最小
           max = 1,  # 最大
           n = 1000) # 乱数の数
hist(x)

In [ ]:
x <- as.integer(      # 整数に変換
  runif(min = 1,      # 最小
        max = 7,      # 最大 + 1
        n = 1000))    # 乱数の数
hist(x, breaks = 0:6) # 結果は割愛

In [ ]:
n <- 100
p <- 0.5
r <- 10000
x <- rbinom(size = n, # 試行回数
            prob = p, # 確率
            n = r)    # 乱数の数
hist(x, breaks = max(x) - min(x))

In [ ]:
r <- 10000
x <- rnorm(mean = 50, # 平均
           sd = 5,    # 標準偏差
           n = r)     # 乱数の数
hist(x, breaks = 40)

In [ ]:
library(tidyverse)

f <- function(k) {
  n <- 10000
  tmp <- replicate(n = n, expr = g(rnorm(n =  k, sd = 3)))
  list(k = k,
       mean = mean(tmp),       # 平均
       se = sd(tmp) / sqrt(n)) # 標準誤差
}

In [ ]:
g <- var
c(10, 20, 30) %>% map_dfr(f)

In [ ]:
g <- sd
c(5, 10, 15, 20) %>% map_dfr(f)

In [ ]:
g <- function(x) {
  n <- length(x)
  sd(x) *
    sqrt((n - 1) / 2) *
    gamma((n - 1) / 2) /
    gamma(n / 2)
}
c(10, 20, 30) %>% map_dfr(f)

## 4.4 統計的推測

In [ ]:
library(exactci)
library(tidyverse)

a <- 0.05                              # 有意水準
binom.exact(x = 2,                     # 当たった回数
            n = 15,                    # くじを引いた回数
            p = 4 / 10,                # 当たる確率（仮説）
            plot = TRUE,               # p値の描画（結果は次項に掲載）
            conf.level = 1 - a,        # 信頼係数（デフォルト）
            tsmethod = "minlike",      # p値の定義
            alternative = "two.sided") # 両側検定（デフォルト）
                                       # 左片側検定なら'less'
                                       # 右片側検定なら'greater'


In [ ]:
t <- 4 / 10               # 当たる確率
n <- 15                   # くじを引いた回数
x <- 0:n                  # 当たった回数
my_pr  <- dbinom(x, n, t) # x回当たる確率
my_pr2 <- dbinom(2, n, t) # 2回当たる確率

my_data <- data.frame(x = x,
                      probability = my_pr,
                      color = my_pr <= my_pr2) # 当たる確率が，2回当たる確率以下

my_data %>% ggplot(aes(x = x, y = probability, color = color)) +
  geom_point(size = 3) +
  geom_linerange(aes(ymin = 0, ymax = probability), ) + # 垂直線
  geom_hline(yintercept = my_pr2) +                     # 水平線
  theme(legend.position = "none")                       # 凡例を表示しない．

In [ ]:
# 前項の結果（再掲）

In [ ]:
# 前項冒頭のコード

In [ ]:
X <- c(32.1, 26.2, 27.5, 31.8, 32.1, 31.2, 30.1, 32.4, 32.3, 29.9,
       29.6, 26.6, 31.2, 30.9, 29.3)
Y <- c(35.4, 34.6, 31.1, 32.4, 33.3, 34.7, 35.3, 34.3, 32.1, 28.3,
       33.3, 30.5, 32.6, 33.3, 32.2)

t.test(x = X, y = Y,
       conf.level = 0.95,         # 信頼係数（デフォルト）
       paired = TRUE,             # 対標本である．
       alternative = "two.sided") # 両側検定（デフォルト）
                                  # 左片側検定なら'less'
                                  # 右片側検定なら'greater'


In [ ]:
t.test(x = X, y = Y,
       paired = FALSE,   # 対標本ではない（デフォルト）．
       var.equal = TRUE, # 等分散を仮定する．仮定しないならFALSE（デフォルト）．
       alternative = "two.sided",
       conf.level = 0.95)


In [ ]:
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/smoker.csv")
my_data <- read_csv(my_url)

In [ ]:
head(my_data)

In [ ]:
my_table <- table(my_data)
my_table

In [ ]:
chisq.test(my_table, correct = FALSE)


In [ ]:
X <- rep(0:1, c(13, 2)) # 手順1
X

tmp <- sample(X, size = length(X), replace = TRUE) # 手順2
tmp

sum(tmp) # 手順3

n <- 10^5
result <- replicate(n, sum(sample(X, size = length(X), replace = TRUE))) # 手順4

In [ ]:
hist(x = result, breaks = 0:15,
     right = FALSE)

In [ ]:
quantile(result, c(0.025, 0.975))